# 1) imports

In [1]:
import findspark
findspark.init()
# Spark:
from pyspark     import SparkConf
from pyspark     import SparkContext
from pyspark.sql import SparkSession

# Python :
from datetime import datetime
import math

# 2) configuration de spark

In [2]:
user           = "romain - Analyse cycliste"
ip_fares       = "54.37.12.49"
master         = "spark://54.37.12.49:7077"
executor_cores = 2 # Ne prendra que des machines qui ont des exécuteurs qui peuvent avoir ce nombre de coeurs
                   # nb de cores par executor (doit être inférieur au nombre de CPU par machine)
nb_cores_max   = 6 # détermine le nombre d'executeur max => max( arrondi.min(nb_cores_max/executor_cores)
parallelism    = 4 * executor_cores
nb_executor    = nb_cores_max//executor_cores
current_date   = datetime.now().strftime("%Y-%m-%d %H:%M")
appName        = "%s le %s"%(user, current_date)
print("il y aura %s executors avec chacun %s coeurs (possiblement sur moins de machines)"%(nb_executor, executor_cores))


il y aura 3 executors avec chacun 2 coeurs (possiblement sur moins de machines)


# 3) création d'un objet "configuration"

In [3]:
dico_conf = {
    "spark.app.name"             : appName        ,
    "spark.master"               : master         ,
    "spark.cores.max"            : nb_cores_max   , 
    "spark.executor.cores"       : executor_cores ,
    "spark.default.parallelism"  : parallelism    , 
    "spark.python.worker.memory" : "1042M"        ,
}
conf = SparkConf()
for k,v in dico_conf.items():
    conf = conf.set(k,v)

# 4) création d'un spark context et d'une spark Session

In [4]:
sc    = SparkContext.getOrCreate(conf=conf)
spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc.applicationId

'app-20180227043024-0389'

# 5) créons une dataframe : 

In [5]:
file_path = "hdfs://54.37.12.49:8020/cyclistes/cycliste_cyclistes.csv"
df        = spark.read.option("header", "true").\
                    option("inferSchema", "true").\
                    csv(file_path)

# 6) changer les strings en int

In [6]:
from pyspark.ml.feature import StringIndexer

In [7]:
string_Indexer = StringIndexer()
string_Indexer.setInputCol("sexe")
string_Indexer.setOutputCol("label")

string_indexer_model = string_Indexer.fit(df)
cyclistes_doubles    = string_indexer_model.transform(df)

# 7) création d'une colonne de vecteurs

In [8]:
from pyspark.ml.feature import VectorAssembler

In [12]:
colonnes

['cycliste', 'sportif', 'age', 'sexe', 'nb_km', 'vitesse', 'attente', 'label']

In [13]:
colonnes             = cyclistes_doubles.columns
colonnes_a_supprimer = ["sexe", "sexe-num", "cycliste", "label"]
colonnes_sans_y      = [col for col in colonnes if col not in colonnes_a_supprimer]

In [14]:
x_y = VectorAssembler()
x_y.setInputCols(colonnes_sans_y)
x_y.setOutputCol("features")
points = x_y.transform(cyclistes_doubles)

In [23]:
colonnes_sans_y

['sportif', 'age', 'nb_km', 'vitesse', 'attente']

In [24]:
points.columns

['cycliste',
 'sportif',
 'age',
 'sexe',
 'nb_km',
 'vitesse',
 'attente',
 'label',
 'features']

# 10) création d'un jeu de test et d'apprentissage

In [25]:
splits              = points.randomSplit([0.8, 0.2])
jeu_d_apprentissage = splits[0].cache()
jeu_de_test         = splits[1].cache()

# 11) création d'un algo (cart)

In [26]:
from pyspark.ml.classification import DecisionTreeClassifier
dt          = DecisionTreeClassifier()
dtmodel     = dt.fit(jeu_d_apprentissage)
predictions = dtmodel.transform(jeu_de_test)

In [27]:
jeu_d_apprentissage.take(1)

[Row(cycliste='cycliste_aze10', sportif=-0.5, age=37, sexe='femme', nb_km=11.050541331, vitesse=19.2038127355, attente=0.5, label=1.0, features=DenseVector([-0.5, 37.0, 11.0505, 19.2038, 0.5]))]

## 11.1) mesure de qualité

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
bceval = BinaryClassificationEvaluator()
bceval.evaluate(predictions)

0.7582058616942723

# 12) enregistrement et loading sur hdfs

In [29]:
output_file = "hdfs://54.37.12.49:8020/cart_model_for_cycliste_v5"
dtmodel.save(output_file)

In [20]:
from pyspark.ml.classification import DecisionTreeClassificationModel
loaded_model = DecisionTreeClassificationModel.load(output_file)